In [7]:
import pandas as pd
import requests
import json
import re
from tqdm import tqdm
import random

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

In [2]:
df = pd.read_csv('banned_book_data_combined_lists.csv')
print(df.shape)

(55278, 8)


In [ ]:
genres = dict()
titles = dict()

with open('titles.csv','w',encoding='utf-8') as out:
    for i,row in tqdm(df.iterrows()):
        title = row['publication']
        title = re.sub(r'\s+','+',title)
        url = f'https://openlibrary.org/search.json?q={title}'
        #print(url)
        headers = {'User-Agent': select_agent() }
        response = requests.get(url, headers=headers)
        if response:
            json_data = response.json()
            if len(json_data['docs'])>0:
                publication = json_data['docs'][0]
                if 'title' in publication:
                    titles[row['publication']] = publication['title']

                if 'subject' in publication:
                    #print(json_data['docs'][0]['subject'])
                    genres[row['publication']] = publication['subject']

742it [1:02:20,  1.42s/it]

In [ ]:
# Convert and write JSON object to file
with open("genres.json", "w") as outfile: 
    json.dump(genres, outfile)